In [15]:
import numpy as np
import pandas as pd
import math 
import urllib.request
import requests 
import altair as alt

# Defining the Function

In [2]:
# This function downloads the desired structure and prepares a new file to the rest of the calculations
def Fetch(PDB_ID: str):
    url = 'https://files.rcsb.org/download/{}.pdb'.format(PDB_ID)
    PDB_file = requests.get(url, allow_redirects=True)
    A = open(PDB_ID  + '.pdb', 'wb').write(PDB_file.content)
    pdbfile = '{}.pdb'.format(PDB_ID)
    file = open(pdbfile, 'r')
    lines = file.readlines()
    string = 'MODEL        1                                                                  \n'
    for count,Element in enumerate(lines):
        if string in Element:
            break
    lines = lines[count:]
   
    with open('{}_refined.pdb'.format(PDB_ID), 'w') as f:
        for l in lines:
            f.write(l)
            if 'END                                                                             ' in l:
                break
    return url

pdb_id = input('please insert the pdb id: ').lower()
PDB = Fetch(pdb_id)


# Reading the generated pdb file

In [3]:
pdb = pd.read_fwf(
    "{}_refined.pdb".format(pdb_id),
    header=None,
    infer_nrows=3000,
)

,0,1,2,3,4,5,6,7,8,9,10,11
0,MODEL,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ATOM,1.0,N,ASP,A,1.0,1.324,-5.770,-9.642,1.0,0.0,N
2,ATOM,2.0,CA,ASP,A,1.0,2.139,-6.771,-10.393,1.0,0.0,C
3,ATOM,3.0,C,ASP,A,1.0,1.242,-7.918,-10.889,1.0,0.0,C
4,ATOM,4.0,O,ASP,A,1.0,0.032,-7.850,-10.787,1.0,0.0,O
...,...,...,...,...,...,...,...,...,...,...,...,...
6007,ATOM,598.0,HG23,VAL,A,40.0,27.079,-0.841,-3.003,1.0,0.0,H
6008,TER,599.0,NaN,VAL,A,40.0,NaN,NaN,NaN,NaN,NaN,NaN
6009,ENDMDL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6010,MASTER,NaN,135,0,NaN,0.0,0.000,0 0,6 30,1.0,0.0,NaN


In [4]:
pdb = pd.read_fwf(
    "{}_refined.pdb".format(pdb_id),
    header=None,
    infer_nrows=2000,
)

col_names = [
    "Atom",
    "row",
    "Atom_id",
    "res_id",
    "strand",
    "res_number",
    "x",
    "y",
    "z",
    "q_num1",
    "q_num2",
    "atom_identifier",
]

snapshot_finder = pdb.iloc[::-1]
snapshot_finder = snapshot_finder.loc[snapshot_finder[0].str.contains("MODEL", case=False)]
snapshot = snapshot_finder[2].astype(int).max()
snapshot
pdb = pdb.dropna()
pdb.reset_index(drop=True, inplace=True)
pdb.columns = col_names
pdb = pdb.astype({'row': int, 'res_number': int, 'x': np.float_, 'y': np.float_, 'z': np.float_})



# Defining Necessary Dataframes

In [5]:
H_Bond_list = pd.DataFrame()

# Counting the number of residues and atoms of the protein

In [16]:
atom_count, residue_count = int(pdb.iloc[::-1].iloc[0][1]), int(pdb.iloc[::-1].iloc[0][5])

40

# Features of the PDB file

In [7]:
features = pd.DataFrame({'Atom number': [atom_count], 'Residue number': residue_count, 'Sanpshot number': snapshot})
features

,Atom number,Residue number,Sanpshot number
0,598,40,10


# Calculation of Hydrogen Bond

In [8]:
for SN in range (0,snapshot): 
   # n = (Num_atom * (step_counter-1))+((step_counter)*9)
  PDB = pdb.loc[(0 + (SN * atom_count)):((atom_count-1) + (SN * atom_count))]
  N_list = PDB.query("Atom_id == 'N'")
  
  N_list = N_list[N_list.row != 1.0]
  N = list(zip(N_list["res_number"], N_list["x"],N_list["y"],N_list["z"]))  
  O_list = PDB.query("Atom_id == 'O'")
  O = list(zip(O_list["res_number"], O_list["x"],O_list["y"],O_list["z"]))
  H_list = PDB.query("Atom_id == 'H'")
  H = list(zip(H_list["res_number"], H_list["x"],H_list["y"],H_list["z"]))

  H_N = list(zip(H_list["res_number"], H_list["x"],H_list["y"],H_list["z"], N_list["res_number"],
                                           N_list["x"],N_list["y"],N_list["z"]))
  for res_num_O, x_O, y_O, z_O  in O:
      for res_num_H, x_H, y_H, z_H, res_num_N, x_N, y_N, z_N in H_N:
        calc_dist = math.dist([x_O,y_O,z_O], [x_N, y_N, z_N])

        if (calc_dist) > 3.2 or (res_num_N==res_num_O):

            continue

        L1 = math.dist([x_N,y_N,z_N], [x_O, y_O, z_O])
        L2 = math.dist([x_N,y_N,z_N], [x_H, y_H, z_H])
        dx1, dy1, dz1 = x_N - x_O, y_N - y_O, z_N - z_O

        dx2, dy2, dz2 = x_N - x_H, y_N - y_H, z_N - z_H

        dot = (dx1 * dx2) + (dy1 * dy2) + (dz1 * dz2)
        Cosine = dot / (L1 * L2)
        Arccos = math.acos(Cosine)
        AngleD = Arccos*180/3.14159265359
        if AngleD > 35:
            continue
        H_Bond_list = H_Bond_list.append({
          'snapshot' : int(SN+1),'res_H' : res_num_H, "res_O" : res_num_O, 'res_N' : res_num_N,
           'distance' : calc_dist,'Angle': AngleD}, ignore_index=True)

        H_Bond_list.to_csv('H_Bond_{}.csv'.format(pdb_id))


# Statistical information on the Hydrogen Bond types

In [9]:
info = pd.read_csv('H_Bond_{}.csv'.format(pdb_id))

info = info.drop(columns=['Unnamed: 0','res_H', 'distance', 'Angle' ])
info = info.groupby(['res_N', 'res_O']).count()

info.to_csv('counting_list_{}.csv'.format(pdb_id))

### General Statistic information

In [10]:
data = pd.read_csv('counting_list_{}.csv'.format(pdb_id))
data.columns = ['Donor', 'Acceptor', 'Total number']
data = data.astype({'Donor': int, 'Acceptor': int, 'Total number': int})
data


,Donor,Acceptor,Total number
0,3,1,2
1,4,2,1
2,5,3,1
3,7,4,1
4,7,5,3
5,8,6,1
6,10,6,1
7,10,7,1
8,14,12,1
9,18,14,6


### Variance

In [11]:
variance = data.var() # Alternatively, ddof=0 can be set to normalize by N instead of N-1:
variance

Donor           136.744939
Acceptor        125.892038
Total number     14.183536
dtype: float64

### Probability of Occurance

In [12]:
probability_Table = pd.DataFrame({'Acceptor': data['Acceptor'], 'Donor': data['Donor'], 
                                  'probability': data['Total number'] / data['Total number'].max() })
probability_Table.to_csv('Hbond_probability_{}.csv'.format(pdb_id))
probability_Table


,Acceptor,Donor,probability
0,1,3,0.2
1,2,4,0.1
2,3,5,0.1
3,4,7,0.1
4,5,7,0.3
5,6,8,0.1
6,6,10,0.1
7,7,10,0.1
8,12,14,0.1
9,14,18,0.6


# Counting the number of Hydrogen bond in each snapshot

In [13]:
H_Bond_list['snapshot'] = H_Bond_list['snapshot'].astype(int)
H_Bond_list = H_Bond_list.groupby(['snapshot']).count().drop(columns=['distance', 'res_N', 'res_H', 'res_O'])
H_Bond_list = H_Bond_list.rename(columns={'Angle': 'H_Bond Count'})
H_Bond_list

,H_Bond Count
snapshot,
1,20
2,21
3,20
4,20
5,18
6,19
7,16
8,22
9,19


In [22]:
H_bond_info = pd.read_csv('H_Bond_1iyt.csv').drop('Unnamed: 0', axis=1)
H_bond_info = H_bond_info.astype({"snapshot": int})
H_bond_info

,snapshot,res_H,res_O,res_N,distance,Angle
0,1,7.0,5.0,7.0,2.749334,19.853513
1,1,10.0,7.0,10.0,2.893319,22.403067
2,1,11.0,7.0,11.0,2.808524,2.868816
3,1,12.0,8.0,12.0,2.746952,18.269477
4,1,15.0,11.0,15.0,3.047366,27.140094
...,...,...,...,...,...,...
242,10,34.0,32.0,34.0,2.912082,34.646513
243,10,36.0,33.0,36.0,2.818941,28.936434
244,10,37.0,33.0,37.0,2.771385,16.166373
245,10,38.0,34.0,38.0,2.721030,17.244363


In [27]:
alt.Chart(H_bond_info).mark_bar().encode(
    x=alt.X('snapshot', bin=alt.Bin(maxbins=residue_count)),
    y='count()',
    color='snapshot'
).properties(
        width=600,
        height=400
    )

alt.Chart(...)

In [28]:
alt.Chart(H_bond_info).mark_point(filled=True, size=400, opacity=0.5).encode(
    x='res_O',
    y='res_H',
).facet('snapshot')

alt.FacetChart(...)

In [30]:
graph = alt.Chart(H_bond_info).mark_rect().encode(
    alt.X('res_O', bin=alt.Bin(maxbins=residue_count), scale=alt.Scale(domain=[0, residue_count])),
    alt.Y('res_N', bin=alt.Bin(maxbins=residue_count)),
    alt.Color('count()')).properties(
        width=600,
        height=400
    )
graph

alt.Chart(...)